# Calculate the area of the enclosed geometric object with psydat file

In [ ]:
from psychopy.misc import fromFile
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import LineString
from shapely.ops import polygonize, unary_union

### Import
A psydat file can be imported using the psychopy `fromFile` function: 
If you want to know the detailed content of the data in psydat file, please check the notebook 'raw_data.ipynb'

In [ ]:
psydata = fromFile("example.psydat")

### Plot of results for each trial
For example, a scatter plot of the mouse positions for each trial, labelled by the condition, trial number and repetition number:

In [ ]:
colors = ["blue", "green", "red", "cyan", "magenta", "yellow", "black", "orange"]

nTrials, nReps = psydata.sequenceIndices.shape
fig, axs = plt.subplots(nTrials, nReps, figsize=(6, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for trial in range(nTrials):
    for rep in range(nReps):
        loc = (trial, rep)
        condition = psydata.sequenceIndices[loc]
        target_radius = psydata.trialList[condition]["target_size"]
        central_target_radius = psydata.trialList[condition]["central_target_size"]
        ax = axs[loc]
        ax.set_title(f"Trial {trial}, Rep {rep} [Condition {condition}]")
        for positions, target_pos, color in zip(
            psydata.data["to_target_mouse_positions"][loc],
            psydata.data["target_pos"][loc],
            colors,
        ):
            ax.plot(positions[:, 0], positions[:, 1], color=color)
            ax.add_patch(
                plt.Circle(
                    target_pos,
                    target_radius,
                    edgecolor="none",
                    facecolor=color,
                    alpha=0.1,
                )
            )
        if not psydata.trialList[condition]["automove_cursor_to_center"]:
            for positions, color in zip(
                psydata.data["to_center_mouse_positions"][loc],
                colors,
            ):
                ax.plot(positions[:, 0], positions[:, 1], color=color)
            ax.add_patch(
                plt.Circle(
                    [0, 0],
                    central_target_radius,
                    edgecolor="none",
                    facecolor="black",
                    alpha=0.1,
                )
            )

plt.show()

### Plot of area calculation results for each trial

How to calculate the area of irregular geometric object with given coordinates from psydat file?

The build-in operation `area` in the library `shapely` can calculate the area of geometry object. However, only for the valid one->not self intersected.

To tackle the self-intersection problem, the strategy is to split one self intersected object into the union of `LineString`(a geometry type composed of one or more line segments), then construct a bunch of valid polygons from these lines, then calculate the area of each valid polygon, sum them up.  

In [ ]:
def plot_and_calculate_area(data):
    nTrials, nReps = data.sequenceIndices.shape
    for trial in range(nTrials):
        for rep in range(nReps):
            loc = (trial, rep)
            condition = data.sequenceIndices[loc]
            central_target_radius = data.trialList[condition]["central_target_size"]

            # if condition "automove_cursor_to_center" is deselected, plot the line to center, fill the enclosed area and output the area
            if not data.trialList[condition]["automove_cursor_to_center"]:
                fig, ax = plt.subplots(1, 1, figsize=(6, 6))
                ax.set_xbound(-0.5, 0.5)
                ax.set_ybound(-0.5, 0.5)
                ax.set_title(
                    f"area plot for Trial {trial}, Rep {rep} [Condition {condition}]"
                )
                print("---------------------------------------------")
                print(
                    "area of Trial %d, Rep %d [Condition %s]" % (trial, rep, condition)
                )
                for (
                    to_target_mouse_positions,
                    to_center_mouse_positions,
                    target_pos,
                    color,
                ) in zip(
                    data.data["to_target_mouse_positions"][loc],
                    data.data["to_center_mouse_positions"][loc],
                    data.data["target_pos"][loc],
                    colors,
                ):
                    coords = np.concatenate(
                        (to_target_mouse_positions, to_center_mouse_positions)
                    )
                    lr_coords = np.concatenate(
                        (coords[:], to_target_mouse_positions[0:1])
                    )
                    lr = LineString(lr_coords)
                    multi_LineString = unary_union(lr)
                    area = 0
                    for pg in polygonize(multi_LineString):
                        area += pg.area
                        ax.plot(*pg.exterior.xy, color=color)
                        ax.fill(*pg.exterior.xy, facecolor=color)

                    ax.add_patch(
                        plt.Circle(
                            target_pos,
                            target_radius,
                            edgecolor="none",
                            facecolor=color,
                            alpha=0.1,
                        )
                    )
                    print("%s, area: %f" % (color, area))

                ax.add_patch(
                    plt.Circle(
                        [0, 0],
                        central_target_radius,
                        edgecolor="none",
                        facecolor="black",
                        alpha=0.1,
                    )
                )

    plt.show()


plot_and_calculate_area(psydata)

### Special cases
This strategy can also be applied to the following special cases:

#### no movement
There is no lines in the plot

In [ ]:
psydata_no_movement = fromFile("example_no_movement.psydat")
plot_and_calculate_area(psydata_no_movement)

#### too much movement
The whole plot is full of lines

In [ ]:
psydata_over_movement = fromFile("example_over_movement.psydat")
plot_and_calculate_area(psydata_over_movement)

#### some targets got reached, some not
For some targets, no line is approaching them, the corresponding area is 0

In [ ]:
psydata_target_not_reached = fromFile("example_target_not_reached.psydat")
plot_and_calculate_area(psydata_target_not_reached)

#### select "Automatically move cursor to center" condition
The trials with condition "Automatically move cursor to center" selected will not be drawn, only the trials with condition "Automatically move cursor to center" deselected will be shown in the plot.

In [ ]:
psydata_select_back_to_center = fromFile("example_select_back_to_center.psydat")
plot_and_calculate_area(psydata_select_back_to_center)

#### not closed line
For the path from the center to target and back to center is not closed, will be closed by the first and last coordinates automatically.

In [ ]:
psydata_not_closed_line = fromFile("example_not_closed_line.psydat")
plot_and_calculate_area(psydata_not_closed_line)